# Project Part 2: Preprocessing & First Model Building and Evaluation

## Getting Images from files

This is the same as in ./data-exploration.ipynb

In [1]:
import os
# define the directories where the imgaes are
pizza_directory="./pizza/"
not_pizza_directory="./not_pizza/"

# get the images from the directories
not_pizza_files=[[0, not_pizza_directory, i] for i in os.listdir(not_pizza_directory)]
pizza_files=[[1, pizza_directory, i] for i in os.listdir(pizza_directory)]

print("Number of pizza images:",len(pizza_files))
print("Number of non-pizza images:",len(not_pizza_files))

# combine the image sets
all_files = not_pizza_files + pizza_files

print("Total number of images:", len(all_files))

# each element in all_files is of the format [is_pizza: boolean, directory: string, filename: string]
# in other words, for each `img` in `all_files`:
#    - img[0] = 0 if not pizza, 1 if pizza
#    - img[1] = directory where the image is
#    - img[2] = file name

Number of pizza images: 983
Number of non-pizza images: 983
Total number of images: 1966


## Preprocessing the data

Resize all images to 512x512 and put them into a new directory.

Create the following arrays:

- **X** - array of the pixel rgb values of the resized images. Each element is in the shape **[512][512][3]**
- **y** - array of whether the corresponding image is of a pizza or not. Each element is a **Boolean**

In [25]:
from PIL import Image

import numpy as np

standardSize = (512,512)
resizedImages = list()
isPizzaArray = list()

# structure is:
# resizedImages[i] = 2-D numpy array of pixels, [512][512][3]


PATH = os.path.abspath(os.getcwd())
if (not os.path.exists(f'{PATH}/resizedImages')):
    os.mkdir('resizedImages')

for i, image in enumerate(all_files):
    imgData = Image.open(image[1] + image[2])
    imgData = imgData.resize(standardSize)
    imgData.save(f'{PATH}/resizedImages/{i}.jpg')
    # storing images
    resizedImages.append(np.array(imgData))
    isPizzaArray.append(image[0])

# ! Takes about 30 seconds...
print("Done")

Done


### Convert resizedImages and y to numpy arrary for ANN

In [26]:
X = np.array(resizedImages)
y = np.array(isPizzaArray)
print("x,y:",len(X),len(y))
print(X.shape,y.shape)

x,y: 1966 1966
(1966, 512, 512, 3) (1966,)


## Our First Model

A very simple model to test if we processed our data correctly and to see how the simple model performs

### Splitting into testing and training dataset

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

print(X_train.shape)
print(X_test.shape)

(1572, 512, 512, 3)
(394, 512, 512, 3)


In [5]:
X_train[0]

array([[[122, 124,  94],
        [123, 125,  95],
        [125, 128,  99],
        ...,
        [165, 144,  97],
        [166, 144,  97],
        [167, 145,  98]],

       [[127, 131,  98],
        [128, 132,  99],
        [130, 133, 103],
        ...,
        [165, 143,  98],
        [166, 144,  97],
        [166, 144,  97]],

       [[136, 139, 105],
        [137, 140, 106],
        [138, 141, 108],
        ...,
        [164, 141,  97],
        [164, 142,  95],
        [164, 142,  95]],

       ...,

       [[108,  97,  59],
        [105,  95,  59],
        [100,  93,  58],
        ...,
        [161,  60,  11],
        [160,  58,   9],
        [159,  57,   8]],

       [[110,  98,  60],
        [107,  96,  60],
        [103,  95,  58],
        ...,
        [160,  56,   8],
        [165,  59,  11],
        [171,  64,  17]],

       [[110,  98,  60],
        [108,  95,  60],
        [103,  95,  58],
        ...,
        [160,  54,   6],
        [169,  63,  15],
        [181,  73,  26]]

### Create the model and fit training data. First svm model, then neural net.

In [6]:
from keras.layers import Conv2D, Flatten, Dense
from keras.models import Sequential
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report
model = Sequential() # Initialising the ANN
# build your convulusional layers here:

imgShape = (512, 512, 3)

model.add(Conv2D(7, 3, input_shape=imgShape, activation = 'tanh'))
model.add(Conv2D(3, 3, activation = 'relu'))
# model.add(Conv2D(2, 3, activation = 'relu'))


# make sure you flatten after you convulusional layers here:
model.add(Flatten())

# Feel free to add hidden layers here: 

# model.add(Dense(units = 10, activation = 'relu'))

# # Hint reduce to a 1d vector as you have in the previous exercises.
model.add(Dense(units = 1, activation = 'sigmoid'))

# compile your model here:

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy')

numEpoches = 2
printOutput = 5

modelData = model.fit(X_train, y_train, batch_size = 15, epochs = numEpoches, verbose = 1)

#! Takes about 5 minutes for 2 epoches...

c:\users\gerri\appdata\local\programs\python\python39\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (None)/charset_normalizer (3.0.1) doesn't match a supported version!
  warnings.warn(


Epoch 1/2
 16/105 [===>..........................] - ETA: 1:40 - loss: 209.1518

KeyboardInterrupt: 

## Classification Report for Testing and Training Data

In [ ]:
from sklearn.metrics import classification_report

yhat = model.predict(X_train)
yhat = [0 if y<=0.5 else 1 for y in yhat]

print('Training Data Classification Report')
print(classification_report(y_train, yhat))

yhat_test = model.predict(X_test)
yhat_test = [0 if y<=0.5 else 1 for y in yhat_test]

print('Test Data Classification Report')
print(classification_report(y_test, yhat_test))

# ! Takes about 45 seconds...

## Where does our model lie on the fitting curve?

We notice that our model overfits our training data, while our model performs slightly better than a coin toss for the testing data.

So our model lies in the fitting curve as shown below

In [ ]:
from IPython.display import Image
Image(filename='fitting_curve.png') 

#! warning, not dark mode friendly

# Second Model, using an SVM.
To determine best svm kernel we iterated through a random subset of our data and computed the score.


In [24]:
from sklearn.svm import SVC
# thing=X_trai[0].reshape(1,X_train[0].size)
# thing.shape
X_trai=X_train[0:10]
y_trai=y_train[0:10]
X_tes=X_test[30:50]
y_tes=y_test[30:50]
X_trai=np.array(X_trai.reshape((X_trai.shape[0], 512*512*3)),dtype=np.uint8)
X_tes=np.array(X_tes.reshape((X_tes.shape[0], 512*512*3)),dtype=np.uint8)

for i in ["linear","rbf","poly","sigmoid"]:
    SVM_cl=SVC(kernel=i)
    SVM_cl.fit(X_trai,y_trai)
    X_tes=X_test[30:50]
    y_tes=y_test[30:50]
    print("Mean accuracy of kernel {} {}".format(i,SVM_cl.score(X_tes,y_tes)))
    #Best appears to be sigmoid

Mean accuracy of kernel linear 0.5
Mean accuracy of kernel rbf 0.6
Mean accuracy of kernel poly 0.5
Mean accuracy of kernel sigmoid 0.7


In [23]:
SVM_cl.predict(X_tes.reshape((X_tes.shape[0], 512*512*3)))

array([0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1])

# Implement SVM calc, with Naive Bayes
We could not append the svm score of each image to the obseravtion data of the images being fed to the neural net. Because the neural net uses convolution, we could not add more dimensions, or replace one of the tensors with the svm score without causing erronious bias. So instead, we inserted the SVM classification and neural net classification into a Categorical Naive Bayes classifier.

In [50]:
thing=model.predict(X_trai)
np.round(thing[1])

1/1 [==============================] - 0s 326ms/step


array([1.], dtype=float32)

In [0]:
from keras.layers import Conv2D, Flatten, Dense
from keras.models import Sequential
from sklearn.svm import SVC
from sklearn.metrics import classification_report
np.random.seed(1)

"Preparing data"
X_trai=X_train[0:20]
y_trai=y_train[0:20]
X_tes=X_test[0:10]
y_tes=y_test[0:10]
"Create and Run NN"
model = Sequential() # Initialising the ANN
imgShape = (512, 512, 3)
model.add(Conv2D(7, 3, activation = 'tanh',input_shape=imgShape))
model.add(Conv2D(3, 2, activation = 'relu'))
model.add(Flatten())
model.add(Dense(1,activation="sigmoid")) #Here is the difference. Each pixel will push the image above or below the SVM constraints.
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy',metrics=["accuracy"])
numEpoches = 3
model.fit(X_trai, y_trai, batch_size = 5, epochs = 3, verbose = 1)
NN_SVM_pred_train=[[np.round(i)] for i in model.predict(X_trai)]
NN_SVM_pred_test=[[np.round(i)] for i in model.predict(X_tes)]

"Create and Run SVM"
X_trai_SVM=np.array(X_trai.reshape((X_trai.shape[0], 512*512*3)),dtype=np.uint8)
X_tes_SVM=np.array(X_tes.reshape((X_tes.shape[0], 512*512*3)),dtype=np.uint8)
SVM_cl=SVC(kernel="sigmoid")
SVM_cl.fit(X_trai_SVM,y_trai)
for i,ii in enumerate(SVM_cl.predict(X_tes_SVM)):
    NN_SVM_pred_test[i].append(ii)
for i,ii in enumerate(SVM_cl.predict(X_trai_SVM)):
    NN_SVM_pred_train[i].append(ii)




In [58]:

"Create and Run Naive"
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import classification_report
CNB_obj=CategoricalNB()                     #initialized object
CNB_obj.fit(X=NN_SVM_pred_train,y=y_trai)
print(classification_report(y_tes,CNB_obj.predict(NN_SVM_pred_test))) #classification report

#! Takes about 5 minutes for 2 epoches...

              precision    recall  f1-score   support

           0       0.50      0.50      0.50         6
           1       0.25      0.25      0.25         4

    accuracy                           0.40        10
   macro avg       0.38      0.38      0.38        10
weighted avg       0.40      0.40      0.40        10



c:\users\gerri\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:845: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = np.asarray(array, order=order)
c:\users\gerri\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:845: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = np.asarray(array, order=order)


In [ ]:

from sklearn.metrics import classification_report
yhat = model.predict(X_train)
yhat = [0 if y<=0.5 else 1 for y in yhat]
print('Training Data Classification Report')
print(classification_report(y_train, yhat))
yhat_test = model.predict(X_test)
yhat_test = [0 if y<=0.5 else 1 for y in yhat_test]

print('Test Data Classification Report')
print(classification_report(y_test, yhat_test))
# ! Takes about 45 seconds...